In [ ]:

#----TURN THESE ON IF RUNNING ON JOVE ON COLAB BY REMOVING THE ''' ... '''

'''
! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
import sys
sys.path.append('./Jove')
sys.path.append('./Jove/jove')
'''

#----TURN THESE ON IF RUNNING ON OWN INSTALLATION (default turned-on)

import sys
sys.path[0:0] = ['../..','../../3rdparty',
                '../../..','../../../3rdparty',
                 '../../../..','../../../../3rdparty'
                ] # Put these at the head of the search path

from jove.DotBashers import *
from jove.Def_DFA import *
from jove.Def_NFA import *
from jove.Def_RE2NFA import *
from jove.Def_NFA2RE import *
from jove.Def_md2mc import *
from jove.JoveEditor import *


# PDA Programming Approaches

* Use stack to tally

* Create alternate denominations

* Use nondeterminism to decide what to match in stack

* Use stack for simple-minded counting

## Stacks to tally

Let Sigma = {a,b}

* Design a PDA for even length strings 
  - as plain DFA
  - Use power of nondet 

* Student exercise: Design a PDA for proper nesting of { (,[,],) }

# As a plain DFA over a,b


In [ ]:
pdaEven1 = md2mc('''PDA
IF : a,''; '' | b,''; '' -> Odd !! Pretend this is an NFA
Odd: a,''; '' | b,''; '' -> IF  !! Ignore the stack, just shuttle between IF and Odd
''')

In [ ]:
dotObj_pda(pdaEven1, FuseEdges=True)

In [ ]:
explore_pda('aaba', pdaEven1)

# As a nondet device

In [ ]:
pdaEven2 = md2mc('''PDA


!! Keep stacking
IF  : a,''; a | b,''; b -> Stk
Stk : a,''; a | b,''; b -> Stk
Stk : '','';''          -> Try !! We guess that we are at the midpoint... could be wrong

Try : a,a; '' | b,b; '' | a,b; '' | b,a; '' -> Try !! Match around the guessed midpoint

Try : '',#; # -> FoundMid  !! The number of items stacked = the number matched -- must be even! If not, fail

''')

In [ ]:
dotObj_pda(pdaEven2, FuseEdges=True)

In [ ]:
explore_pda('aaba', pdaEven2, STKMAX=10)

In [ ]:
explore_pda('', pdaEven2, STKMAX=10)

In [ ]:
explore_pda('aabaa', pdaEven2, STKMAX=10)

In [ ]:
explore_pda('aba', pdaEven2, STKMAX=10)

In [ ]:
explore_pda('abaabb', pdaEven2, STKMAX=10)

In [ ]:
# Students match ([...]) or [(...)]
# Try this, students

## Create alternate denominations

Let Sigma = {a,b}
* Design a PDA for #a = 2 #b + 3 #c

aa b
aaa c
aa aaa bc
in some order
a versus b/c game

multiply b's twofold with B
multiply c's threefold with C
then a's and B's and a's and C's are equals

All orders must be heeded

a first - stack - a on top, b next -> BB -> leave B on stack
b first - stack BB - c next stack CCC 



In [ ]:
pdaabc = md2mc('''PDA

!! Stack a,b,c but expand b as BB and c as CCC
IF : a,''; a | b,''; BB | c,''; CCC -> Work 

!! Continue stacking, with the "currency conversion" going on
Work: a,a; aa | b,B; BBB | b,C; BBC | c,B; CCCB | c,C; CCCC | a,#; a# | b,#; BB# | c,#; CCC# -> Work

!! When a b comes, see if we can match upto two a's
Work: b,a; '' -> Undera !! An a may have come and we may have put it on the stack; if so find it at the top

!! Lucky that another  was found; eat that and go back to Work
Undera : '',a; '' -> Work

!! Not another 'a'; so split the earlier seen B and also put back what was popped now; then go to Work
Undera : '',B; BB | '',C; BC | '',#; B#  -> Work

!! When a 'c' comes, see if we can find upto 3 a's on the stack
Work: c,a; '' -> Underaa

!! Two a's are desired underneath; if so, we can knock-off the c; else stack the remaining C's and go Work
Underaa: '', a; '' -> UnderOneMorea

!! we have two unmet a's, and the C's are stacked for that
Underaa: '', B; CCB | '', C; CCC | '',#; CC# -> Work 

!! Avoid stacking anything as we got c versus 3 a's
UnderOneMorea: '',a; '' -> Work 

!! One unmet a for which we stacked a C
UnderOneMorea: '',B; CB | '',C; CC | '',#; C# -> Work 

!! We tally a and B on an equal footing; same with a versus C
Work: a,B; ''| a,C; '' -> Work

!! In the work state, when # shows up, we are ready to accept
Work: '',#; # -> F


''')

In [ ]:
dotObj_pda(pdaabc, FuseEdges=True)

In [ ]:
explore_pda('aaac', pdaabc)

In [ ]:
explore_pda('aaca', pdaabc)

In [ ]:
explore_pda('acaa', pdaabc)

In [ ]:
explore_pda('caaa', pdaabc)

In [ ]:
explore_pda('aaaccaaa', pdaabc)

In [ ]:
explore_pda('aaccaaa', pdaabc)

In [ ]:
explore_pda('aaaccaaabaaaab', pdaabc)

## Stack nondet

a=b or a=c

i.e. {a^i b^j c^k : i=j or i=k }

*consult the book for this*

This is the Sipser example discussed in the book

## Stack for simple counting 

a^i b^j c^k d^l : if i=2 then j=k else l > k

Try the reverse also


In [ ]:
aibjck = md2mc('''PDA
!! First get two a's and then if that happens, look for a bc match else look for a d>c situation
!! Start in state I and not IF because epsilon is not in the language

!! Now d>c must be true
I: c,''; c -> NowCsDs

I: d,''; '' -> F1

I: a,'';''    -> Onea

Onea: a,'';'' -> Twoa
Twoa: a,'';'' -> MoreThanTwoa

Twoa: c,''; c -> NowCDMatch

NowCDMatch: c,'';c -> NowCDMatch

NowCDMatch: d,c; '' -> NowCDMatch

NowCDMatch: '',#; # -> F2

!! d>c when one a only
Onea: d,'';'' -> F1
Onea: c,'';c  -> NowCsDs

!! d>c must hold
MoreThanTwoa: a,'';'' -> MoreThanTwoa

MoreThanTwoa: c,''; c -> NowCsDs

MoreThanTwoa: d,''; '' -> F1

NowCsDs: c,''; c -> NowCsDs

NowCsDs: d,c; '' -> NowDsOnly

NowDsOnly: d,d;'' -> NowDsOnly

NowDsOnly: d,c; '' -> NowDsOnly

NowDsOnly: d,#; # -> F1

MoreThanTwoa: d,'';'' -> F1
F1 : d,''; '' -> F1



''')

In [ ]:
dotObj_pda(aibjck, FuseEdges=True)

In [ ]:
explore_pda('d', aibjck)

In [ ]:
explore_pda('dd', aibjck)

In [ ]:
explore_pda('cdd', aibjck)

In [ ]:
explore_pda('ccdd', aibjck)

In [ ]:
explore_pda('ccd', aibjck)

In [ ]:
explore_pda('acd', aibjck)

In [ ]:
explore_pda('aacd', aibjck)

In [ ]:
explore_pda('aaad', aibjck)

In [ ]:
explore_pda('aaaccdd', aibjck)

In [ ]:
explore_pda('aaaccddd', aibjck)